In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
#default_exp search

In [16]:
#export
from food.tools import *
from food.paths import *
from food.psql import *
import requests
from food.qdrant import *
import torch
from torch.nn import CosineSimilarity
cos = CosineSimilarity(dim=1, eps=1e-08)
import numpy as np

In [17]:
#export
## temp while api is not aperational
# from food.clipmodel import *
from PIL import Image

In [18]:
# #export
# foods = read_sql('foods')
# # foods = foods.drop(columns = ['clip'])
# foods = foods.set_index('id')

In [19]:
#export
collection_name = 'food_prompted'
table = 'foods_prompted'
foods = read_sql(table)
foods = foods.set_index('id')

In [20]:
#export
def read_image_from_url(url=None):
    response = requests.get(url, stream=True)
    return Image.open(response.raw)


In [21]:
#export
def search_image_(url=None,head = 1):
    image_clip = requests.post(f'https://guru.skynet.center/image2vector/?url={url}').json()
    results = client.search(collection_name=collection_name,query_vector=image_clip,top=head)
    image_clip = torch.Tensor(image_clip)
    df = foods.loc[[r.id for r in results]].copy()
    df['score'] = [r.score for r in results]
    df = df.sort_values('score',ascending=False)

    return image_clip,df.reset_index()

series2tensor = lambda series:torch.tensor([np.array(c) for c in series.values])

In [22]:
#export
drop_vector =      lambda clip, i: torch.cat([clip[0:i], clip[i+1:]])

def multiply_vector(clip,i,n=1):
    duplicated = torch.cat([clip[i].reshape(1,768) for _ in range(n)])
    return torch.cat([clip, duplicated.reshape(n,768)])

In [23]:
#export
def multiple_foods(url):

    image_clip,selected = search_image_(url,head=100)
    selected=selected.reset_index(drop=True)
    clip = series2tensor(selected['clip'])
    initscore = cos(image_clip.reshape(1,768), clip.mean(0).reshape(1,768))

    startscore = initscore.detach().clone()-0.0000001

    while startscore !=initscore:
        startscore = initscore.detach().clone()
        
        selected = selected.reset_index(drop=True)
        for i in reversed(selected.index):
            clip = series2tensor(selected['clip'])
            dropped = drop_vector(clip,i)
            testscore = cos(image_clip.reshape(1,768), dropped.mean(0).reshape(1,768))

            if testscore > initscore-0.0001: 
                r = selected.loc[i,'text']
                selected = selected.drop(i)
                initscore = testscore

            else:
                extra = multiply_vector(clip,i,1)
                testscore = cos(image_clip.reshape(1,768), extra.mean(0).reshape(1,768))
                if testscore > initscore: 
                    selected = selected.append(selected.loc[i])
                    initscore = testscore


            

        print(initscore)
    count = selected.groupby('text')['clip'].count().sort_index()
    r = selected.drop_duplicates('text').set_index("text").sort_index()
    r['count'] = count
    r = r.sort_values('count',ascending = False)
    r['cumcount'] = r['count'].cumsum()/r['count'].sum()

    description = '. '.join(r[r['cumcount']<0.9]['description'].unique().tolist())


    return r.drop(columns = ['clip']), description, selected

In [24]:
#export
def search_image(url):
    r, desc, sel = multiple_foods(url)
    df = sel[['energy','protein','carb','fat']].mean().to_frame().T
    df['description'] = desc
    return df.round()


In [25]:
search_image(url)

tensor([0.2988], dtype=torch.float64)
tensor([0.3053], dtype=torch.float64)
tensor([0.3056], dtype=torch.float64)
tensor([0.3056], dtype=torch.float64)
tensor([0.3057], dtype=torch.float64)
tensor([0.3057], dtype=torch.float64)


,energy,protein,carb,fat,descriptioin
0,221.333333,10.0475,27.095833,8.1025,"Hamburger slider, from fast food. Potato, hash..."


In [28]:
!nbdev_build_lib

Converted 00_clipmodel.ipynb.
Converted 00_custom_pandas.ipynb.
Converted 00_paths.ipynb.
Converted 00_psql.ipynb.
Converted 00_qdrant.ipynb.
Converted 00_tools.ipynb.
Converted 01_multiple_foods.ipynb.
Converted 01_multiple_foods_segmantation.ipynb.
Converted 01_search.ipynb.
Converted 0_template copy 2.ipynb.
Converted 0_template.ipynb.
Converted OFA.ipynb.
Converted Untitled-1.ipynb.
Converted foodd dataset.ipynb.
Converted foods_prompted_tosql.ipynb.
Converted inference.ipynb.
Converted multiple3105.ipynb.
Converted multiple_3005.ipynb.
No export destination, ignored:
#export
def search_image(url=None,head = 1):
    image_clip = requests.post(f'https://guru.skynet.center/image2vector/?url={url}').json()
    results = client.search(collection_name=collection_name,query_vector=image_clip,top=head)
    image_clip = torch.Tensor(image_clip)
    df = foods.loc[[r.id for r in results]].copy()
    df['score'] = [r.score for r in results]
    df = df.sort_values('score',ascending=False)

 